In [1]:
from gridworld.env import GridWorldEnv

pygame 2.5.2 (SDL 2.28.3, Python 3.10.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
env = GridWorldEnv(size=10)

In [8]:
env.action_space

Discrete(4)

In [10]:
env.observation_space

Dict('agent': Box(0, 9, (2,), int64), 'target': Box(0, 9, (2,), int64))

In [13]:
env.reward_range

(-inf, inf)